In [ ]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *

In [ ]:
plates = [
    # "111_20240131",
    # "179_20240219",
    # "199_20240130",
    "393_20231129",
    # "401_20231123",
    # "404_20231127",
    # "407_20240102"
]
directory_targ = directory_project
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
timesteps = {
    "111_20240131": 60,
    "179_20240219": 5,
    "199_20240130": 57,
    "401_20231123": 25,
    "404_20231127": 19,
    "393_20231129": 30,
    "407_20240102": 64,
}

In [ ]:
folders = all_folders.loc[all_folders["unique_id"] == "393_20231129"]
folders = folders.sort_values(by="datetime")
folders = folders.iloc[30:31]

exp = Experiment(directory_targ)
exp.load(folders)
exp.dates.sort()
for t in range(len(folders)):
    exp.load_tile_information(t)

In [ ]:
load_graphs(exp, directory_targ)
exp.save_location = ""

load_study_zone(exp)

In [ ]:
import networkx as nx

G = exp.nx_graph[t]
components = nx.connected_components(G)

# Find the largest connected component
largest_component = max(components, key=len)

# Create a new graph representing the largest connected component
largest_component_graph = G.subgraph(largest_component)
exp.nx_graph[t] = largest_component_graph

In [ ]:
edges = get_all_edges(exp, t)
nodes = get_all_nodes(exp, t)
nodes_source = [
    node
    for node in nodes
    if not is_in_study_zone(node, t, 1000, 100)[1]
    and is_in_study_zone(node, t, 1000, 100)[0]
]
nodes_sink = [
    node
    for node in nodes
    if is_in_study_zone(node, t, 1000, 150)[1] and node.degree(t) == 1
]

In [ ]:
nodes_source_label = [485364, 1369923, 1358718, 232037, 1329603, 1361585, 1361408]
nodes_source_label = [195492, 211492, 217447, 227, 241, 311, 165239]
nodes_source_label = [903, 906]  # 179
nodes_source2 = [Node(label, exp) for label in nodes_source_label]

In [ ]:
ax = plot_full(exp, t, nodes=nodes_source)
# ax.scatter(
#     spore_datatable["y"] / 5, spore_datatable["x"] / 5, s=20, color="blue", alpha=0.4
# )

In [ ]:
f = lambda edge: np.log(
    edge.width(t) * edge.length_um(t) * edge.end.degree(t) * edge.begin.degree(t)
)
edges = get_all_edges(exp, t)
values = [f(edge) for edge in edges]
weights = [edge.length_um(t) for edge in edges]

In [ ]:
edges = get_all_edges(exp, t)
nodes = get_all_nodes(exp, t)
nodes_source = [
    node
    for node in nodes
    if not is_in_study_zone(node, t, 1000, 100)[1]
    and is_in_study_zone(node, t, 1000, 100)[0]
]
nodes_source = nodes_source
# nodes_sink = [
#     node
#     for node in nodes
#     if is_in_study_zone(node, t, 1000, 150)[1]
# ]
nodes_sink = [
    node
    for node in nodes
    if is_in_study_zone(node, t, 1000, 150)[1]
    if node.degree(t) == 1
]
weights = {(edge.begin.label, edge.end.label): edge.length_um(t) for edge in edges}
nx.set_edge_attributes(exp.nx_graph[t], weights, "length")
weights = {(edge.begin.label, edge.end.label): 1 / edge.length_um(t) for edge in edges}
nx.set_edge_attributes(exp.nx_graph[t], weights, "1/length")
t = 0
G = exp.nx_graph[t]
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
len_connected = [len(nx_graph.nodes) for nx_graph in S]
final_current_flow_betweeness = {}
final_betweeness = {}

for g in S:
    source = [node.label for node in nodes_source if node.label in g]
    sink = [node.label for node in nodes_sink if node.label in g]
    # current_flow_betweeness = nx.edge_current_flow_betweenness_centrality_subset(
    #     g, source, sink, weight="1/length"
    # )
    # betweeness = nx.edge_current_flow_betweenness_centrality_subset(
    #     g, sink, source, weight="length"
    # )

    betweeness = nx.edge_betweenness_centrality_subset(
        g, source, sink, normalized=True, weight="length"
    )
    # for edge in current_flow_betweeness.keys():
    #     final_current_flow_betweeness[edge] = current_flow_betweeness[edge]
    for edge in betweeness.keys():
        final_betweeness[edge] = betweeness[edge]

for edge in exp.nx_graph[t].edges:
    # if (
    #     edge not in final_current_flow_betweeness.keys()
    #     and (edge[1], edge[0]) not in final_current_flow_betweeness.keys()
    # ):
    #     final_current_flow_betweeness[edge] = 0
    if (
        edge not in final_betweeness.keys()
        and (edge[1], edge[0]) not in final_betweeness.keys()
    ):
        final_betweeness[edge] = 0
nx.set_edge_attributes(exp.nx_graph[t], final_betweeness, "betweenness")

In [ ]:
vmax = 12
vmin = 2
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    cmap=cm.get_cmap("cool", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    figsize=(12, 7),
)

In [ ]:
plt.close("all")
vmax = -2
vmin = -4
ax = plot_edge_color_value(
    exp,
    t,
    lambda edge: np.log10(edge.betweeness(t)),
    cmap=cm.get_cmap("autumn", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    figsize=(12, 7),
)

In [ ]:
edges = get_all_edges(exp, t)
BCs = [edge.betweeness(t) for edge in edges]
lengths = [edge.length_um(t) for edge in edges]
diameters = [edge.length_um(t) for edge in edges]
data = pd.DataFrame({"BC": BCs, "length": lengths, "diameter": diameters})
data["logBC"] = np.log10(data["BC"])

In [ ]:
from scipy.stats import linregress

fig, ax = plt.subplots(figsize=(4, 4))  # Adjust 'width' and 'height' as needed
data = pd.DataFrame({"BC": BCs, "length": lengths, "diameter": diameters})

data = data.loc[data["BC"] > 1e-5]

data["logBC"] = np.log10(data["BC"])
ax.hist(data["logBC"], bins=12, weights=data["length"], density=True)
data = data.loc[data["BC"] > 1e-4]

counts, bins = np.histogram(data["logBC"], bins=8, weights=data["length"], density=True)
# ax.set_yscale('log')
# ax.set_xscale('log')

# Prepare data for regression
# Calculate bin centers from edges
bin_centers = 0.5 * (bins[1:] + bins[:-1])
# Perform linear regression on the log-transformed bin centers and counts
slope, intercept, r_value, p_value, std_err = linregress(bin_centers, np.log(counts))

# Create a line based on the regression to overlay
x = np.linspace(min(bin_centers), max(bin_centers), 100)
y = np.exp(intercept + slope * x)

# Plot the regression line
ax.plot(x, y, "r-", label=f"Linear fit: slope={slope:.2f}")
ax.set_yscale("log")
ax.set_ylabel("density")
ax.set_xlabel("$log_{10}(BC)$")

In [ ]:
timesteps

In [ ]:
plates = [
    "111_20240131",
    "179_20240219",
    "199_20240130",
    "393_20231129",
    "401_20231123",
    "404_20231127",
    "407_20240102",
]
data_whole = pd.DataFrame()
for plate in plates:
    timestep = timesteps[plate]
    folders = all_folders.loc[all_folders["unique_id"] == plate]
    folders = folders.sort_values(by="datetime")
    folders = folders.iloc[timestep : timestep + 1]

    exp = Experiment(directory_targ)
    exp.load(folders)
    exp.dates.sort()
    for t in range(len(folders)):
        exp.load_tile_information(t)
    load_graphs(exp, directory_targ)
    exp.save_location = ""

    load_study_zone(exp)

    G = exp.nx_graph[t]
    components = nx.connected_components(G)

    # Find the largest connected component
    largest_component = max(components, key=len)

    # Create a new graph representing the largest connected component
    largest_component_graph = G.subgraph(largest_component)
    exp.nx_graph[t] = largest_component_graph
    edges = get_all_edges(exp, t)
    nodes = get_all_nodes(exp, t)
    nodes_source = [
        node
        for node in nodes
        if not is_in_study_zone(node, t, 1000, 100)[1]
        and is_in_study_zone(node, t, 1000, 100)[0]
    ]
    nodes_source = nodes_source
    nodes_sink = [
        node
        for node in nodes
        if is_in_study_zone(node, t, 1000, 150)[1]
        if node.degree(t) == 1
    ]
    weights = {(edge.begin.label, edge.end.label): edge.length_um(t) for edge in edges}
    nx.set_edge_attributes(exp.nx_graph[t], weights, "length")
    weights = {
        (edge.begin.label, edge.end.label): 1 / edge.length_um(t) for edge in edges
    }
    nx.set_edge_attributes(exp.nx_graph[t], weights, "1/length")
    t = 0
    G = exp.nx_graph[t]
    S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
    len_connected = [len(nx_graph.nodes) for nx_graph in S]
    final_current_flow_betweeness = {}
    final_betweeness = {}

    for g in S:
        source = [node.label for node in nodes_source if node.label in g]
        sink = [node.label for node in nodes_sink if node.label in g]
        # current_flow_betweeness = nx.edge_current_flow_betweenness_centrality_subset(
        #     g, source, sink, weight="1/length"
        # )
        # betweeness = nx.edge_current_flow_betweenness_centrality_subset(
        #     g, sink, source, weight="length"
        # )

        betweeness = nx.edge_betweenness_centrality_subset(
            g, source, sink, normalized=True, weight="length"
        )
        # for edge in current_flow_betweeness.keys():
        #     final_current_flow_betweeness[edge] = current_flow_betweeness[edge]
        for edge in betweeness.keys():
            final_betweeness[edge] = betweeness[edge]

    for edge in exp.nx_graph[t].edges:
        # if (
        #     edge not in final_current_flow_betweeness.keys()
        #     and (edge[1], edge[0]) not in final_current_flow_betweeness.keys()
        # ):
        #     final_current_flow_betweeness[edge] = 0
        if (
            edge not in final_betweeness.keys()
            and (edge[1], edge[0]) not in final_betweeness.keys()
        ):
            final_betweeness[edge] = 0
    nx.set_edge_attributes(exp.nx_graph[t], final_betweeness, "betweenness")
    edges = get_all_edges(exp, t)
    BCs = [edge.betweeness(t) for edge in edges]
    lengths = [edge.length_um(t) for edge in edges]
    diameters = [edge.length_um(t) for edge in edges]
    ids = [plate] * len(diameters)
    data = pd.DataFrame(
        {"BC": BCs, "length": lengths, "diameter": diameters, "unique_id": ids}
    )
    data["logBC"] = np.log10(data["BC"])
    data_whole = pd.concat((data_whole, data))
    # break

In [ ]:
data_whole.to_csv("BC_dataA5.csv")

In [ ]:
from scipy.stats import linregress
import numpy as np
import matplotlib.pyplot as plt

# Example data_whole and plates setup
# data_whole = ...
# plates = [...]

# Determine the size of the subplot grid
n = len(plates)
cols = 4  # max number of columns
rows = n // cols + (1 if n % cols else 0)  # calculate rows needed

fig, axs = plt.subplots(
    rows, cols, figsize=(cols * 4, rows * 4)
)  # Adjust 'width' and 'height' as needed
fig.tight_layout(pad=3.0)  # Adjust spacing between subplots
slopes = []
for idx, plate in enumerate(plates):
    data = data_whole.loc[data_whole["unique_id"] == plate]
    data["logBC"] = np.log10(data["BC"])

    data = data.loc[data["BC"] > 1e-10]
    counts, bins = np.histogram(
        data["logBC"], bins=10, weights=data["length"], density=True
    )

    miniBC = np.min(bins[np.where(counts > 0.02)])
    peakBC = bins[np.argmax(counts)]
    data = data.loc[data["BC"] > 10**miniBC]

    # Get current subplot row and column
    row = idx // cols
    col = idx % cols
    if rows == 1:  # Single row
        ax = axs[col]
    elif cols == 1:  # Single column
        ax = axs[row]
    else:
        ax = axs[row, col]

    ax.hist(data["logBC"], bins=12, weights=data["length"], density=True)
    data = data.loc[data["BC"] > 10**peakBC]

    counts, bins = np.histogram(
        data["logBC"], bins=10, weights=data["length"], density=True
    )
    bin_centers = 0.5 * (bins[1:] + bins[:-1])
    slope, intercept, r_value, p_value, std_err = linregress(
        bin_centers, np.log(counts)
    )
    x = np.linspace(min(bin_centers), max(bin_centers), 100)
    y = np.exp(intercept + slope * x)
    slope, intercept, r_value, p_value, std_err = linregress(
        bin_centers, np.log10(counts)
    )
    slopes.append(slope)
    ax.plot(x, y, "r-", label=f"Linear fit: slope={slope:.2f}")
    ax.set_yscale("log")
    ax.set_ylabel("density")
    ax.set_xlabel("$log_{10}(BC)$")
    ax.legend()

# Adjust layout for any empty subplots if the number of plates is not a multiple of 'cols'
for empty_ax in axs.flat[idx + 1 :]:
    empty_ax.axis("off")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
data = data_whole.loc[data_whole["unique_id"] == "401_20231123"]
data["logBC"] = np.log10(data["BC"])

data = data.loc[data["BC"] > 1e-10]
counts, bins = np.histogram(
    data["logBC"], bins=10, weights=data["length"], density=True
)

miniBC = np.min(bins[np.where(counts > 0.02)])
peakBC = bins[np.argmax(counts)]
data = data.loc[data["BC"] > 10**miniBC]
ax.hist(data["logBC"], bins=12, weights=data["length"], density=True)
data = data.loc[data["BC"] > 10**peakBC]

counts, bins = np.histogram(
    data["logBC"], bins=10, weights=data["length"], density=True
)
bin_centers = 0.5 * (bins[1:] + bins[:-1])
slope, intercept, r_value, p_value, std_err = linregress(bin_centers, np.log(counts))
x = np.linspace(min(bin_centers), max(bin_centers), 100)
y = np.exp(intercept + slope * x)
slope, intercept, r_value, p_value, std_err = linregress(bin_centers, np.log10(counts))

ax.plot(x, y, "r-", label=f"Linear fit: slope={slope:.2f}")
ax.set_yscale("log")
ax.set_ylabel("density")
ax.set_xlabel("$log_{10}(BC)$")
ax.legend()